In [0]:
# !pip install eli5

In [25]:
import pickle
from IPython.display import display, HTML
from sklearn.base import BaseEstimator, ClassifierMixin
import eli5
from eli5.lime import TextExplainer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import BertTokenizer, BertModel, GPT2Model, GPT2Tokenizer

In [26]:
with open ('../data/fake', 'rb') as fp:
    fake = pickle.load(fp)

with open ('../data/real', 'rb') as fp:
    real = pickle.load(fp)

In [35]:
MODEL_NAME = 'lstm_gpt.pth'

# LSTM with GPT-2 embeddings

In [28]:
pretrained_weights = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_weights)
model = GPT2Model.from_pretrained(pretrained_weights)

In [29]:
embeddings_pretrained = model.get_input_embeddings()
embeddings_pretrained

Embedding(50257, 768)

In [30]:
EMBEDDINGS_DIM = embeddings_pretrained.embedding_dim
VOCAB_SIZE = embeddings_pretrained.num_embeddings
EMB_PRETRAINED = True

In [31]:
def tokenize(text, tokenizer=tokenizer):
    return tokenizer.encode(text)

In [32]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, 
                 emb_pretrained, embeddings):
        super(MyModel, self).__init__()
        self.emb_pretrained = emb_pretrained
        self.embedding =  embeddings if self.emb_pretrained else nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True,
                          )
        
        self.fc = nn.Linear(hidden_size * 2 *2, 1)
    def forward(self, x):
        
        x = self.embedding(x)
           
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = torch.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [39]:
config = {'tokenization/embeddings': 'gpt2',
            'batch_size': 256,
          'hidden_size' : 256,
            'num_epochs': 10}

In [40]:
model = MyModel(VOCAB_SIZE,
                embed_size=EMBEDDINGS_DIM,
                hidden_size=config['hidden_size'],
                emb_pretrained = EMB_PRETRAINED,
                embeddings = embeddings_pretrained
               )
# model.to(device)

In [41]:
results = torch.load('../model_chechpoints/{}'.format(MODEL_NAME) , map_location=torch.device('cpu'))
model.load_state_dict(results['model_state_dict'])

<All keys matched successfully>

In [48]:
class LSTMClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, model):
        super(LSTMClassifier, self).__init__() 
        self.model = model
        self.classes_ = (0,1)

    def fit(self, X=None, y=None, **kwargs):
        return self

    def predict_proba(self, texts):
        """
        texts: list of texts
        :return: ndarray n_texts x n_classes
        """
    
        ids = [tokenizer.encode(text) for text in texts]
        
        for ind, el in enumerate(ids):
            if len(el) < len(ids[0]):
                while len(el) < len(ids[0]):
                    el.append(1)
            if len(el) > len(ids[0]):
                ids[ind] = el[:len(ids[0])]            
        
        tensor = torch.tensor(ids)
        self.model.eval()
        with torch.no_grad():
            logits = self.model.forward(tensor)
        sigmoids = torch.sigmoid(logits)  # First predict the 'Real' prob
        opposite_class_prob = 1 - sigmoids  # Then calculate the 'Fake' prob
        probs = torch.cat((opposite_class_prob, sigmoids), dim=-1)
        
        return probs.detach().numpy()

    def predict(self, text):
        return int(torch.round(self.predict_proba(text)).item())

In [49]:
model_estimator = LSTMClassifier(model)
model_estimator.fit()

LSTMClassifier(model=MyModel(
  (embedding): Embedding(50257, 768)
  (rnn): LSTM(768, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=1, bias=True)
))

In [50]:
model_estimator.predict_proba([fake[0]])

array([[0.9671992, 0.0328008]], dtype=float32)

In [70]:
text = fake[223]
text
print(model_estimator.predict_proba([text]))
te = TextExplainer(random_state=42, position_dependent=True, n_samples=2000)
te.fit(doc=text, predict_proba=model_estimator.predict_proba)
te.show_prediction(target_names=['Fake','Real'])

[[0.8299799 0.1700201]]


In [71]:
text = real[1]
text
print(model_estimator.predict_proba([text]))
te = TextExplainer(random_state=42, position_dependent=True, n_samples=2000)
te.fit(doc=text, predict_proba=model_estimator.predict_proba)
te.show_prediction(target_names=['Fake','Real'])

[[0.02061832 0.9793817 ]]


In [72]:
for i in range(0, 50, 10):
    text_fake = fake[i]
    text_real = real[i]
    te = TextExplainer(random_state=42, position_dependent=True, n_samples=1000)
    print(model_estimator.predict_proba([text_fake]))
    te.fit(doc=text_fake, predict_proba=model_estimator.predict_proba)
    print('True label: Fake')
    display(te.show_prediction(target_names=['Fake','Real']))
    
    te = TextExplainer(random_state=42, position_dependent=True, n_samples=1000)
    print(model_estimator.predict_proba([text_real]))
    te.fit(doc=text_real, predict_proba=model_estimator.predict_proba)
    print('True label: Real')
    display(te.show_prediction(target_names=['Fake','Real']))

[[0.9671992 0.0328008]]
True label: Fake


[[0.01165509 0.9883449 ]]
True label: Real


[[0.9754403  0.02455968]]
True label: Fake


[[0.02225822 0.9777418 ]]
True label: Real


[[0.98931336 0.01068661]]
True label: Fake


[[0.456039 0.543961]]
True label: Real


[[0.8539853 0.1460147]]
True label: Fake


[[0.69299006 0.3070099 ]]
True label: Real


[[0.91327345 0.08672655]]
True label: Fake


[[0.05012804 0.94987196]]
True label: Real
